In [5]:
#Her importerer vi modulene som trengs til dette
import json, requests

#Denne printer bare litt penere...
from pprint import pprint

In [9]:
#Forbered etterspørsel
#Her er brukt som eksempel "Gruppering av årstrinn i grunnskolen 2014-11 - variant av Årstrinn i grunnskolen 2014-11"

url = 'http://data.ssb.no/api/klass/v1/variants/798/'
headers = {'Accept': 'application/json'}

#Gjør etterspørsel
r = requests.get(url, headers = headers)

#Av en eller annen grunn returnerer KLASS APIen json som bytestring...
#La oss decode...
cont = r.content.decode('utf8').replace("'", '"')

#Konverter til en liste vi kan bruke
cont = json.loads(cont)

#Print ut resultat
#pprint(cont)

In [10]:
#Særbehandling ift. innhold i eksempelrequest
#Om du henter annen data fra Klass så må dette sikkert skrives om

#Tittel og info
print(cont['name'])
print('Eies av: %s' % (cont['owningSection']) )
print('Kontaktperson: %s\nEmail: %s\n' % (cont['contactPerson']['name'], cont['contactPerson']['email']))

#Loop gjennom alle klassifikasjoner og print elementer med innhold underveis
for item in cont['classificationItems']:
    code = item['code']
    parentCode = item['parentCode']
    level = item['level']
    name = item['name']
    print( f'Kode: {code}')
    
    #Av og til var denne tom, så da printer vi den ikke
    if parentCode != "":
        print( f'Foreldrekode: {parentCode}')
    
    print( f'Nivå: {level} \nNavn: {name}\n')

Gruppering av årstrinn i grunnskolen 2014-11  - variant av Årstrinn i grunnskolen 2014-11
Eies av: 360 - Seksjon for utdannings og kulturstatistikk
Kontaktperson: Ekren, Rachel
Email: Rachel.Ekren@ssb.no

Kode: 1-4
Nivå: 1 
Navn: 1.-4. trinn

Kode: 1-10
Nivå: 1 
Navn: Elever alle trinn

Kode: 5-7
Nivå: 1 
Navn: 5.-7. trinn

Kode: 8-10
Nivå: 1 
Navn: 8.-10. trinn

Kode: 01
Foreldrekode: 1-4
Nivå: 2 
Navn: 1. trinn

Kode: 02
Foreldrekode: 1-4
Nivå: 2 
Navn: 2. trinn

Kode: 03
Foreldrekode: 1-4
Nivå: 2 
Navn: 3. trinn

Kode: 04
Foreldrekode: 1-4
Nivå: 2 
Navn: 4. trinn

Kode: 05
Foreldrekode: 5-7
Nivå: 2 
Navn: 5. trinn

Kode: 06
Foreldrekode: 5-7
Nivå: 2 
Navn: 6. trinn

Kode: 07
Foreldrekode: 5-7
Nivå: 2 
Navn: 7. trinn

Kode: 08
Foreldrekode: 8-10
Nivå: 2 
Navn: 8. trinn

Kode: 09
Foreldrekode: 8-10
Nivå: 2 
Navn: 9. trinn

Kode: 10
Foreldrekode: 8-10
Nivå: 2 
Navn: 10. trinn



In [8]:
#Ta ut til json igjen om ønskelig
s = json.dumps(cont, indent = 4 , sort_keys=True)
print(s)

{
    "_links": {
        "self": {
            "href": "http://data.ssb.no/api/klass/v1/variants/798"
        }
    },
    "changelogs": [],
    "classificationItems": [
        {
            "code": "1-4",
            "level": "1",
            "name": "1.-4. trinn",
            "notes": null,
            "parentCode": "",
            "shortName": null
        },
        {
            "code": "1-10",
            "level": "1",
            "name": "Elever alle trinn",
            "notes": null,
            "parentCode": "",
            "shortName": null
        },
        {
            "code": "5-7",
            "level": "1",
            "name": "5.-7. trinn",
            "notes": null,
            "parentCode": "",
            "shortName": null
        },
        {
            "code": "8-10",
            "level": "1",
            "name": "8.-10. trinn",
            "notes": null,
            "parentCode": "",
            "shortName": null
        },
        {
            "code": "01",
